In [4]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 7982
WEBHOOK_BASE = ""
ADMIN_URL = "http://the_wallet-agent:7981"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL)


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7ff3582372b0>


IPython autoawait is `on`, and set to use `asyncio`


In [5]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)


In [6]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Attributes: {attributes}")
    
cred_listener = {
    "topic": "issue_operation_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)

Subscribing too: issue_operation_credential


Listening webhooks failed! OSError(98, "error while attempting to bind on address ('0.0.0.0', 7982): address already in use") occurred.


In [4]:
response = await agent_controller.issuer.get_records()
print(response)

{'results': [{'credential_exchange_id': '2df8b4d0-036e-499a-97ca-f280a64818b6', 'schema_id': 'PEcmuSQ7G3z771J7dse1R5:2:Operation:0.0.1', 'auto_issue': False, 'updated_at': '2023-01-05 09:28:40.562913Z', 'connection_id': 'fc385e08-b61e-4f85-96fe-68b2116d5a5d', 'state': 'offer_received', 'credential_offer': {'schema_id': 'PEcmuSQ7G3z771J7dse1R5:2:Operation:0.0.1', 'cred_def_id': 'PEcmuSQ7G3z771J7dse1R5:3:CL:42:default', 'key_correctness_proof': {'c': '42708810129381310593731787285905928927780621002191234228031353474026709822978', 'xz_cap': '37087548935742989125417637171356295772318137931046285299356114644428111180830539376120045214373423481671450372399435514755314641572140478767555138501745473888615860406653009354400747858737972635912822665960281767789516290293571211399335727034751475463363415411337555898121477412424847882788458452579527379028995088168155078711052853168729441354553870158222039972037071208424832797133834588277019110290288701438423400689097516849676919384533308256596121750

In [7]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")

Credential exchange 2df8b4d0-036e-499a-97ca-f280a64818b6, role: holder, state: offer_received
Being offered: [{'name': 'type', 'value': 'drug_A'}, {'name': 'dose', 'value': '10'}, {'name': 'speed', 'value': '10'}, {'name': 'volt', 'value': '10'}]


In [8]:
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 2df8b4d0-036e-499a-97ca-f280a64818b6, role: holder, state: request_sent


In [9]:
response = await agent_controller.issuer.store_credential(cred_ex_id, "Operation 0")
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 2df8b4d0-036e-499a-97ca-f280a64818b6, role: holder, state: credential_acked


### You can get  credential by your credential id ref(seller notebook)

In [10]:
response = await agent_controller.terminate()
print(response)

None
